In [1]:
import numpy as np
import torch
from torch.utils import data
from torch import nn
from d2l import torch as d2l

In [177]:
batch_size = 2
true_w = torch.tensor([3.22,-4.3])
true_b = 4

features = torch.normal(0,1,[256*5,len(true_w)])
labels = torch.matmul(features,true_w) + true_b
# labels = labels.reshape(-1,1)
features.shape,labels.shape

(torch.Size([1280, 2]), torch.Size([1280]))

In [178]:
def get_dataloader(is_train=True):
    dataset = data.TensorDataset(features, labels)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

dataloader = get_dataloader()
X,Y = next(iter(dataloader))
X,Y

(tensor([[ 0.6207, -0.3851],
         [-0.4232, -0.7980]]),
 tensor([7.6547, 6.0689]))

In [179]:
net = nn.Sequential(nn.Linear(len(true_w),1))
net[0].weight.data.normal_(3,1)
net[0].bias.data.zero_()

loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
net,net[0].weight.data,net[0].bias.data

(Sequential(
   (0): Linear(in_features=2, out_features=1, bias=True)
 ),
 tensor([[2.7848, 2.1390]]),
 tensor([0.]))

In [180]:
num_epochs = 3
dataloader = get_dataloader()

for epoch in range(num_epochs):
    for X, y in dataloader:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

C:\ProgramData\Miniconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\ProgramData\Miniconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1280])) that is different to the input size (torch.Size([1280, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1, loss 35.870026
epoch 2, loss 35.100548
epoch 3, loss 36.140697


In [173]:
loss(net(X),Y),loss(net(X),Y.reshape(-1,1))

(tensor(53.6752, grad_fn=<MseLossBackward0>),
 tensor(23.4886, grad_fn=<MseLossBackward0>))

In [181]:
net(X)-Y, net(X)-Y.reshape(-1,1)

(tensor([[-6.1678, -4.5821],
         [-2.8263, -1.2406]], grad_fn=<SubBackward0>),
 tensor([[-6.1678],
         [-1.2406]], grad_fn=<SubBackward0>))